In [1]:
import os
import logging
from argparse import ArgumentParser
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import libpysal
import math
import matplotlib.pyplot as plt

logging.getLogger("lightning").setLevel(logging.WARNING)

In [2]:
from pigvae.trainer import PLGraphAE
from pigvae.synthetic_graphs.hyperparameter import add_arguments
from pigvae.synthetic_graphs.data import GraphDataModule, EvalRandomBinomialGraphDataset
from pigvae.synthetic_graphs.metrics import Critic

In [3]:
import numpy as np
import torch
from torch.utils.data import Dataset
from torch.utils.data.distributed import DistributedSampler
import random
import pytorch_lightning as pl
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx
import networkx as nx
import networkx.generators as nxg
from networkx.algorithms.shortest_paths.dense import floyd_warshall_numpy
from dataclasses import dataclass
import torch.nn as nn
from torchvision import transforms
from torchvision import datasets

In [4]:
mnist = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=transforms.ToTensor(),
)

In [5]:
SIZE = 24
PATCH_SIZE = 4
class CellDataset(Dataset):

    def __init__(self, imgs, targets, channels, img_transform=None):

        super().__init__()
        self.img_transform = img_transform
        self.imgs = imgs
        self.targets = targets
        self.channels = channels

    def __getitem__(self, idx):
        
        img = self.imgs[idx][self.channels, :].to(torch.float32)
        target = self.targets[idx]
        if self.img_transform:
            img = self.img_transform(img.unsqueeze(0))

        return img, target

    def __len__(self):
        return len(self.imgs)
    
class SplitPatches(nn.Module):
    
    def __init__(self, patch_size):
        super().__init__()
        self.patch_size = patch_size
        self.unfold = torch.nn.Unfold(kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        # x -> B c h w
        # bs, c, h, w = x.shape
        bs, c, h, w = x.shape
        
        x = self.unfold(x)
        # x -> B (c*p*p) L
        
        # Reshaping into the shape we want
        a = x.view(bs, c, self.patch_size, self.patch_size, -1).permute(0, 4, 1, 2, 3)
        a = a.view(bs, -1, c *self.patch_size* self.patch_size)
        # a -> ( B no.of patches c p p )
        return a
     
train_transform = transforms.Compose([
                                    transforms.Resize((SIZE, SIZE)),
                                    SplitPatches(PATCH_SIZE)
                                     ])


In [6]:
cell_dataset = CellDataset(mnist.data.unsqueeze(1), mnist.targets, [0], train_transform)

In [7]:
class GridGraphDataset(Dataset):
    def __init__(self,
                 grid_size: int,
                 imgs,
                 targets, 
                 channels, 
                 img_transform=None,
                 sample_per_epoch: int = 10_000
        ):
        self.grid_size = grid_size
        self.sample_per_epoch = sample_per_epoch
        self.img_transform = img_transform
        self.imgs = imgs[:self.sample_per_epoch]
        self.targets = targets[:self.sample_per_epoch]
        self.channels = channels
        
    def __len__(self):
        return self.sample_per_epoch
    
    def __getitem__(self, idx):
        g = nx.grid_graph((self.grid_size, self.grid_size))
        img = self.imgs[idx][self.channels, :].to(torch.float32)
        target = self.targets[idx]
        if self.img_transform:
            img = self.img_transform(img.unsqueeze(0))
        return (g, img, target)

class GeometricGraphDataset(Dataset):
    def __init__(self, n_min=12, n_max=20, samples_per_epoch=100000, **kwargs):
        super().__init__()
        self.n_min = n_min
        self.n_max = n_max
        self.samples_per_epoch = samples_per_epoch

    def __len__(self):
        return self.samples_per_epoch

    def __getitem__(self, idx):
        n = np.random.randint(low=self.n_min, high=self.n_max)
        g = nxg.geometric.random_geometric_graph(n=n, radius=0.5)
        return g


class BinomialGraphDataset(Dataset):
    def __init__(self, n_min=12, n_max=20, p_min=0.4, p_max=0.6,
                 samples_per_epoch=100000, pyg=False, **kwargs):
        super().__init__()
        self.n_min = n_min
        self.n_max = n_max
        self.p_min = p_min
        self.p_max = p_max
        self.samples_per_epoch = samples_per_epoch
        self.pyg = pyg

    def __len__(self):
        return self.samples_per_epoch

    def get_largest_subgraph(self, g):
        g = g.subgraph(sorted(nx.connected_components(g), key=len, reverse=True)[0])
        g = nx.convert_node_labels_to_integers(g, first_label=0)
        return g

    def __getitem__(self, idx):
        n = np.random.randint(low=self.n_min, high=self.n_max)
        if self.p_min == self.p_max:
            p = self.p_min
        else:
            p = np.random.uniform(low=self.p_min, high=self.p_max)
        g = nxg.random_graphs.binomial_graph(n, p)
        if self.pyg:
            g = from_networkx(g)
        return g

class DenseGraphBatch(Data):
    def __init__(self, node_features, edge_features, mask, **kwargs):
        super().__init__(**kwargs)  # Call the parent class' constructor (Data)
        self.node_features = node_features
        self.edge_features = edge_features
        self.mask = mask

    @classmethod
    def from_sparse_graph_list(cls, data_list, labels=True):
        if labels:
            max_num_nodes = max([graph.number_of_nodes() for graph, _, _ in data_list])
        else:
            max_num_nodes = max([graph.number_of_nodes() for graph in data_list])
        node_features = [] 
        edge_features = []
        mask = []
        y = []
        props = []
        for graph, embedding, label in data_list:
            y.append(label)
            num_nodes = graph.number_of_nodes()
            props.append(torch.Tensor([num_nodes]))
            graph.add_nodes_from([i for i in range(num_nodes, max_num_nodes)])
            nf = embedding
            node_features.append(nf)
            # dm = torch.from_numpy(floyd_warshall_numpy(graph)).long()
            # dm = torch.clamp(dm, 0, 5).unsqueeze(-1)
            # dm = torch.zeros((max_num_nodes, max_num_nodes, num_nodes)).type_as(dm).scatter_(2, dm, 1).float()
            # edge_features.append(dm)
            mask.append((torch.arange(max_num_nodes) < num_nodes).unsqueeze(0))
        node_features = torch.cat(node_features, dim=0)
        # edge_features = torch.stack(edge_features, dim=0)
        edge_features = torch.tensor(edge_features)
        mask = torch.cat(mask, dim=0)
        props = torch.cat(props, dim=0)
        # Instead of using cls, use Data directly for initialization
        batch = DenseGraphBatch(node_features=node_features, edge_features=edge_features, mask=mask, properties=props)
        if labels:
            batch.y = torch.Tensor(y)
        return batch
    
# class DenseGraphDataLoader(torch.utils.data.DataLoader):
#     def __init__(self, dataset, batch_size=1, shuffle=False, labels=True, **kwargs):    
#         super().__init__(dataset, batch_size, shuffle,
#                          collate_fn=lambda data_list: DenseGraphBatch.from_sparse_graph_list(data_list, labels), **kwargs)

def dense_graph_collate_fn(data_list):
    return DenseGraphBatch.from_sparse_graph_list(data_list)


class DenseGraphDataLoader(torch.utils.data.DataLoader):
    def __init__(self, dataset, batch_size=1, shuffle=False, labels=True, **kwargs):
        self.labels = labels
        super().__init__(
            dataset,
            batch_size=batch_size,
            shuffle=shuffle,
            collate_fn=dense_graph_collate_fn,  # Directly pass the standalone function
            **kwargs,
        )


class GraphDataModule(pl.LightningDataModule):
    def __init__(
            self, 
            graph_family,
            graph_kwargs=None, 
            samples_per_epoch=100000, 
            batch_size=32,
            distributed_sampler=True, 
            num_workers=1
        ):
        super().__init__()
        if graph_kwargs is None:
            graph_kwargs = {}
        self.graph_family = graph_family
        self.graph_kwargs = graph_kwargs
        self.samples_per_epoch = samples_per_epoch
        self.num_workers = num_workers
        self.batch_size = batch_size
        self.distributed_sampler = distributed_sampler
        self.train_dataset = None
        self.eval_dataset = None
        self.train_sampler = None
        self.eval_sampler = None

    def make_dataset(self, samples_per_epoch):
        if self.graph_family == "binomial":
            ds = BinomialGraphDataset(samples_per_epoch=samples_per_epoch, **self.graph_kwargs)
        elif self.graph_family == 'grid':
            ds = GridGraphDataset(sample_per_epoch=samples_per_epoch, **self.graph_kwargs) 
        else:
            raise NotImplementedError
        return ds
    
    def train_dataloader(self):
        self.train_dataset = self.make_dataset(samples_per_epoch=self.samples_per_epoch)
        if self.distributed_sampler:
            train_sampler = DistributedSampler(
                dataset=self.train_dataset,
                shuffle=False
            )
        else:
            train_sampler = None
        
        return DenseGraphDataLoader(
            dataset=self.train_dataset,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            pin_memory=True,
            sampler=train_sampler,
        )

    def val_dataloader(self):
        self.eval_dataset = self.make_dataset(samples_per_epoch=4096)
        if self.distributed_sampler:
            eval_sampler = DistributedSampler(
                dataset=self.eval_dataset,
                shuffle=False
            )
        else:
            eval_sampler = None
        return DenseGraphDataLoader(
            dataset=self.eval_dataset,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            pin_memory=True,
            sampler=eval_sampler,
        )

In [8]:
graph_kwargs_binomial = {
        "n_min": 6,
        "n_max": 8,
        "m_min": 1,
        "m_max": 5,
        "p_min": 0.4,
        "p_max": 0.6
    }

graph_kwargs_grid = {
        "grid_size": 6,
        "imgs": mnist.data.unsqueeze(1),
        "targets": mnist.targets,
        "img_transform": train_transform,
        "channels": [0]
}


graph_family_binomial = "binomial"
graph_family_grid = "grid"


datamodule = GraphDataModule(
        graph_family=graph_family_grid,
        graph_kwargs=graph_kwargs_grid,
        batch_size=32,
        num_workers=0,
        samples_per_epoch=1000,
        distributed_sampler=None
    )

In [9]:
data = datamodule.train_dataloader()

In [10]:
batches = [el for el in data]

# My implementation 

In [83]:
import scipy.sparse as scs
import numpy as np
import torch
from libpysal.weights import lat2W


class LaplacianGridEmbedding(nn.Module):
    def __init__(
        self, 
        grid_size: int, 
        out_dim: int, 
    ):
        super().__init__()
        self.grid_size = grid_size
        self.out_dim = out_dim

        # Define the fully connected layer
        self.fc1 = nn.Linear(grid_size**2, out_dim)

    def forward(self, x: torch.Tensor):
       
        x = x.transpose(-2, -1)
        output = self.fc1(x)
        return output
    
    @classmethod
    def get_graph_laplacian(cls, A):
        D = np.diag(A.sum(axis=0))
        L = D - A
        return L

# Define grid size
grid_size = 3  # Example

# Precompute adjacency matrix
A = lat2W(grid_size, grid_size, rook=True).full()[0]
A_sparse = scs.csr_matrix(A)  # Sparse format for efficient Laplacian computation

# Precompute Laplacian
L = LaplacianGridEmbedding.get_graph_laplacian(A)

# Precompute eigen decomposition
eigenvals, eigenvecs = np.linalg.eigh(L)
sorted_eigenvecs = eigenvecs[:, np.argsort(eigenvals)]
sorted_eigenvecs = torch.tensor(sorted_eigenvecs, dtype=torch.float32)

In [84]:
spectralEmbeddings = LaplacianGridEmbedding(
    grid_size=grid_size, 
    out_dim=16
)

In [86]:
spectralEmbeddings(sorted_eigenvecs).shape

torch.Size([9, 16])

In [98]:
def restore_tensor(a, bs, c, h, w, patch_size):
    # Step 1: Reshape a to match the patch grid layout
    a = a.view(bs, -1, c, patch_size, patch_size)
    # a -> (B, num_patches, C, patch_size, patch_size)
    
    # Step 2: Reshape back to (B, C, H, W) by folding the patches
    # Calculate the grid size (L) which is the number of patches in each row and column
    grid_size = int(a.size(1) ** 0.5)  # Assumes square grid (height = width for the patches)
    
    # Unfold back into the original image size
    a = a.view(bs, grid_size, grid_size, c, patch_size, patch_size)
    # a -> (B, grid_size, grid_size, C, patch_size, patch_size)
    
    # Step 3: Permute and reshape to get back to the image format (B, C, H, W)
    x_reconstructed = a.permute(0, 3, 1, 4, 2, 5).contiguous()
    # x_reconstructed -> (B, C, grid_size, patch_size, grid_size, patch_size)
    
    x_reconstructed = x_reconstructed.view(bs, c, h, w)
    # x_reconstructed -> (B, C, H, W)
    
    return x_reconstructed

In [99]:
x = restore_tensor(batches[0].node_features, 32, 1, 24, 24, 4)

In [100]:
from PIL import Image

Image.fromarray(np.array(x[2].reshape(24, 24), dtype=np.uint8))

In [11]:
from dataclasses import dataclass, asdict
import os

DEFAULT_SAVE_DIR = os.path.join(os.getcwd())

@dataclass
class Params:
    id: int = 0
    num_epochs: int = 10
    num_eval_samples: int = 8192
    eval_freq: int = 1000
    save_dir: str = DEFAULT_SAVE_DIR
    precision: int = 32
    progress_bar: bool = True
    test: bool = False
    resume_ckpt: str = ""
    batch_size: int = 32
    lr: float = 0.00005
    kld_loss_scale: float = 0.001
    perm_loss_scale: float = 0.5
    property_loss_scale: float = 0.1
    vae: bool = True
    num_node_features: int = 36
    num_edge_features: int = 36
    emb_dim: int = 64
    graph_encoder_hidden_dim: int = 256
    graph_encoder_k_dim: int = 64
    graph_encoder_v_dim: int = 64
    graph_encoder_num_heads: int = 16
    graph_encoder_ppf_hidden_dim: int = 1024
    graph_encoder_num_layers: int = 16
    graph_decoder_hidden_dim: int = 256
    graph_decoder_k_dim: int = 64
    graph_decoder_v_dim: int = 64
    graph_decoder_num_heads: int = 16
    graph_decoder_ppf_hidden_dim: int = 1024
    graph_decoder_pos_emb_dim: int = 64
    graph_decoder_num_layers: int = 16
    property_predictor_hidden_dim: int = 256
    num_properties: int = 1
    num_workers: int = 1
    shuffle: int = 1
    graph_family: str = "grid"
    n_min: int = 12
    n_max: int = 20
    p_min: float = 0.4
    p_max: float = 0.6
    m_min: int = 1
    m_max: int = 5

# Example usage
hparams = Params()

# Convert to dictionary
params_dict = asdict(hparams)

print(params_dict)


{'id': 0, 'num_epochs': 10, 'num_eval_samples': 8192, 'eval_freq': 1000, 'save_dir': '/Users/tomeknocon/MIM/Master_Thesis/PIGVAE/pigvae/synthetic_graphs', 'precision': 32, 'progress_bar': True, 'test': False, 'resume_ckpt': '', 'batch_size': 32, 'lr': 5e-05, 'kld_loss_scale': 0.001, 'perm_loss_scale': 0.5, 'property_loss_scale': 0.1, 'vae': True, 'num_node_features': 36, 'num_edge_features': 36, 'emb_dim': 64, 'graph_encoder_hidden_dim': 256, 'graph_encoder_k_dim': 64, 'graph_encoder_v_dim': 64, 'graph_encoder_num_heads': 16, 'graph_encoder_ppf_hidden_dim': 1024, 'graph_encoder_num_layers': 16, 'graph_decoder_hidden_dim': 256, 'graph_decoder_k_dim': 64, 'graph_decoder_v_dim': 64, 'graph_decoder_num_heads': 16, 'graph_decoder_ppf_hidden_dim': 1024, 'graph_decoder_pos_emb_dim': 64, 'graph_decoder_num_layers': 16, 'property_predictor_hidden_dim': 256, 'num_properties': 1, 'num_workers': 1, 'shuffle': 1, 'graph_family': 'grid', 'n_min': 12, 'n_max': 20, 'p_min': 0.4, 'p_max': 0.6, 'm_min':

In [109]:
critic = Critic
model = PLGraphAE(params_dict, critic)

In [110]:
lr_logger = LearningRateMonitor()
tb_logger = TensorBoardLogger(hparams.save_dir + "/run{}/".format(hparams.id))
critic = Critic
model = PLGraphAE(params_dict, critic)
graph_kwargs_grid = {
        "grid_size": 6,
        "imgs": mnist.data.unsqueeze(1),
        "targets": mnist.targets,
        "img_transform": train_transform,
        "channels": [0]
}
datamodule = GraphDataModule(
    graph_family=hparams.graph_family,
    graph_kwargs=graph_kwargs_grid,
    batch_size=hparams.batch_size,
    num_workers=hparams.num_workers,
    samples_per_epoch=10_000,
    distributed_sampler=False
)
trainer = pl.Trainer(
    val_check_interval=hparams.eval_freq if not hparams.test else 100,
    accelerator="cpu",
    gradient_clip_val=0.1,
    precision=hparams.precision,
    max_epochs=hparams.num_epochs,
)
trainer.fit(model=model, datamodule=datamodule)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name     | Type    | Params | Mode 
---------------------------------------------
0 | graph_ae | GraphAE | 50.7 M | train
1 | critic   | Critic  | 0      | train
---------------------------------------------
50.7 M    Trainable params
0         Non-trainable params
50.7 M    Total params
202.743   Total estimated model params size (MB)
487       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/tomeknocon/opt/anaconda3/envs/pigvae/lib/python3.8/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/tomeknocon/opt/anaconda3/envs/pigvae/lib/python3.8/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'GridGraphDataset' on <module '__main__' (built-in)>


RuntimeError: DataLoader worker (pid(s) 9619) exited unexpectedly

In [39]:
# datamodule = GraphDataModule(
#         graph_family=graph_family,
#         graph_kwargs=graph_kwargs,
#         batch_size=32,
#         num_workers=0,
#         samples_per_epoch=1000,
#         distributed_sampler=None
#     )
# trainer = pl.Trainer(
#     val_check_interval=hparams.eval_freq if not hparams.test else 100,
#     accelerator="cpu",
#     gradient_clip_val=0.1,
#     precision=hparams.precision,
#     max_epochs=hparams.num_epochs,
# )
# trainer.fit(model=model, datamodule=datamodule)

In [57]:
class DenseGraphBatch1(Data):
    def __init__(self, node_features, edge_features, mask):
        self.node_features = node_features
        self.edge_features = edge_features
        self.mask = mask

    # @classmethod
    # def from_sparse_graph_list(cls, data_list, labels=False):
    #     if labels:
    #         max_num_nodes = max([graph.number_of_nodes() for graph, label in data_list])
    #     else:
    #         max_num_nodes = max([graph.number_of_nodes() for graph in data_list])
    #     node_features = []
    #     edge_features = []
    #     mask = []
    #     y = []
    #     props = []
    #     for data in data_list:
    #         if labels:
    #             graph, label = data
    #             y.append(label)
    #         else:
    #             graph = data
    #         num_nodes = graph.number_of_nodes()
    #         props.append(torch.Tensor([num_nodes]))
    #         graph.add_nodes_from([i for i in range(num_nodes, max_num_nodes)])
    #         nf = torch.ones(max_num_nodes, 1)
    #         node_features.append(nf.unsqueeze(0))
    #         dm = torch.from_numpy(floyd_warshall_numpy(graph)).long()
    #         dm = torch.clamp(dm, 0, 5).unsqueeze(-1)
    #         num_nodes = dm.size(1)
    #         dm = torch.zeros((num_nodes, num_nodes, 6)).type_as(dm).scatter_(2, dm, 1).float()
    #         edge_features.append(dm)
    #         mask.append((torch.arange(max_num_nodes) < num_nodes).unsqueeze(0))
    #     node_features = torch.cat(node_features, dim=0)
    #     edge_features = torch.stack(edge_features, dim=0)
    #     mask = torch.cat(mask, dim=0)
    #     props = torch.cat(props, dim=0)
    #     batch = cls(node_features=node_features, edge_features=edge_features, mask=mask, properties=props)
    #     if labels:
    #         batch.y = torch.Tensor(y)
    #     return batch

    # def __repr__(self):
    #     repr_list = ["{}={}".format(key, list(value.shape)) for key, value in self.__dict__.items()]
    #     return "DenseGraphBatch({})".format(", ".join(repr_list))

In [173]:
import numpy as np


CIRCLE_SIZE = 24


def get_circle_graph(size=CIRCLE_SIZE):
    circle = np.zeros((size, size))
    for i in range(size):
        circle[i, (i + 1) % size] = 1
        circle[i, (i - 1) % size] = 1
    return circle


def get_graph_laplacian(graph):
    d = np.diag(graph.sum(axis=0))
    return d - graph

In [ ]:
def plot_eigen(eigen):
    n = np.sqrt(len(eigen))
    circle_points = np.meshgrid(np.arange(n), np.arange(n), indexing='ij')
    eigen_std = eigen.std()
    vmin=eigen.min() - 1e-3
    vmax=eigen.max() + 1e-3
    plt.scatter(
        x=circle_points[0],
        y=circle_points[1],
        c=eigen,
        vmin=vmin,
        vmax=vmax,
        cmap='seismic',
        s=100,
    )
    plt.colorbar()


def plot_eigens(eigens, eigenvals):
    plot_size = math.ceil(eigens.shape[0] ** 0.5)
    plt.figure(figsize=(5 * plot_size, 5 * plot_size))
    for eigen_nb, eigen_ in enumerate(eigens.T):
        plt.subplot(plot_size, plot_size, eigen_nb + 1)
        plot_eigen(eigen_)
        plt.title(f'Eigen {eigen_nb}: {round(eigenvals[eigen_nb], 2)}')

In [176]:
eigenvals, eigenvecs = np.linalg.eig(L)
sorted_eigenvecs = eigenvecs[:, eigenvals.argsort()]
sorted_eigenvals = eigenvals[eigenvals.argsort()]

In [177]:
eigenvals, eigenvecs = np.linalg.eig(L)
sorted_eigenvecs = eigenvecs[:, eigenvals.argsort()]
sorted_eigenvals = eigenvals[eigenvals.argsort()]

In [179]:
len(sorted_eigenvals)

36

In [180]:
len(sorted_eigenvecs)

36

In [216]:
edges = [...]  # List of edges in the grid graph
k = 36
edge_features_1 = {}
grid = np.meshgrid(np.arange(6), np.arange(6))
for i in range(6):
    for j in range(6):
        feature = eigenvecs[:, :k][:, i] - eigenvecs[:, :k][:, j]  # Use first k eigenvectors
        edge_features_1[(i, j)] = feature

In [258]:
edge_features_tensor = eigenvecs[:, :k][:, None, :] - eigenvecs[:, :k][None, :, :]  #